In [2]:
import pandas as pd
import json
from tqdm import tqdm
import datetime
import os

class JsonParser:

    def __init__(self, path=None, df=None):
        # You might either initialize with a path or with an existing DataFrame.
        self.path = path
        if df is not None:
            self.df = df
        elif path:
            self.df = self.parse_json_file()
        else:
            self.df = pd.DataFrame()

    def parse_json_file(self):
        with open(self.path, "r") as f:
            data = json.load(f)

        gameData = data["gameData"]
        liveData = data["liveData"]["plays"]["allPlays"]

        gameId = gameData.get("game", {}).get("pk", None)
        season = gameData.get("game", {}).get("season", None)
        gameType = gameData.get("game", {}).get("type", None)
        gameDate = datetime.datetime.strptime(gameData.get("datetime", {}).get("dateTime", ""), "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d") if gameData.get("datetime", {}).get("dateTime", None) else None
        homeTeam = gameData.get("teams", {}).get("home", {}).get("abbreviation", None)
        awayTeam = gameData.get("teams", {}).get("away", {}).get("abbreviation", None)

        rows = []
        base_data = {
            "gameId": gameId,
            "season": season,
            "gameType": gameType,
            "gameDate": gameDate,
            "homeTeam": homeTeam,
            "awayTeam": awayTeam,
        }

        for play in liveData:
            row_data = base_data.copy()

            if play["result"]["eventTypeId"] in ["GOAL", "SHOT"]:
                period = play.get("about", {}).get("period", None)
                periodTime = play.get("about", {}).get("periodTime", None)
                byTeam = play.get("team", {}).get("triCode", None)
                eventType = play.get("result", {}).get("eventTypeId", None)
                shotType = play.get("result", {}).get("secondaryType", None)
                coordinates = (play.get("coordinates", {}).get("x", None), 
                               play.get("coordinates", {}).get("y", None))
                strength = play.get("result", {}).get("strength", {}).get("code", None)
                emptyNet = play.get("result", {}).get("emptyNet", None)

                shooterName = None
                goalieName = None  

                for player in play.get("players", []):
                    if player.get("playerType", "") in ["Scorer", "Shooter"]:
                        shooterName = player.get("player", {}).get("fullName", None)
                    elif player.get("playerType", "") == "Goalie":
                        goalieName = player.get("player", {}).get("fullName", None)
            
                row_data.update({
                    "period": period,
                    "periodTime": periodTime,
                    "byTeam": byTeam,
                    "eventType": eventType,
                    "coordinates": coordinates,
                    "shooterName": shooterName,
                    "goalieName": goalieName,
                    "strength": strength,
                    "emptyNet": emptyNet,
                })

                rows.append(row_data)

        return pd.DataFrame(rows)

    def __add__(self, other):
        if not isinstance(other, JsonParser):
            raise ValueError("Can only add JsonParser objects.")
        combined_df = pd.concat([self.df, other.df], ignore_index=True)
        return JsonParser(df=combined_df)

    @staticmethod
    def load_all_seasons():
        if os.path.exists("clean_data.csv"):
            df = pd.read_csv("clean_data.csv")
            print("DataFrame loaded from 'clean_data.csv'")
            return df

        base_parser = JsonParser()

        all_seasons = os.listdir("./data/")
        for season in tqdm(all_seasons, desc="Processing seasons"):
            base_path = f"./data/{season}"
            all_games = os.listdir(base_path)
            for game in tqdm(all_games, desc=f"Processing games in {season}", leave=False):
                parser = JsonParser(base_path + f"/{game}")
                base_parser = base_parser + parser

        base_parser.df.to_csv("clean_data.csv", index=False)
        print("DataFrame saved to 'clean_data.csv'")
        return base_parser.df

In [3]:
import json

with open("./data/2016/2016020001.json", "r") as f:
    data = json.load(f)

In [14]:
shotType = play.get("result", {}).get("secondaryType", None)

["result"]["secondaryType"]

'Wrist Shot'

In [1]:
import pandas as pd

df = pd.read_csv("clean_data.csv")
df.head()

,GAME_ID,SEASON,GAME_TYPE,GAME_DATE,HOME_TEAM,AWAY_TEAM,PERIOD,PERIOD_TIME,BY,EVENT_TYPE,COORDINATES,SHOOTER_NAME,GOALIE_NAME,STRENGTH,EMPTY_NET
0,2016020001,20162017,R,2016-10-12,OTT,TOR,1,01:11,TOR,SHOT,"(-77, 5)",Mitchell Marner,Craig Anderson,NaN,NaN
1,2016020001,20162017,R,2016-10-12,OTT,TOR,1,02:53,OTT,SHOT,"(86, 13)",Chris Kelly,Frederik Andersen,NaN,NaN
2,2016020001,20162017,R,2016-10-12,OTT,TOR,1,04:01,OTT,SHOT,"(23, -38)",Cody Ceci,Frederik Andersen,NaN,NaN
3,2016020001,20162017,R,2016-10-12,OTT,TOR,1,04:46,OTT,SHOT,"(33, -15)",Erik Karlsson,Frederik Andersen,NaN,NaN
4,2016020001,20162017,R,2016-10-12,OTT,TOR,1,06:46,TOR,SHOT,"(-34, 28)",Martin Marincin,Craig Anderson,NaN,NaN
